In [1]:
from sklearn.model_selection import train_test_split
from gensim.models.word2vec import Word2Vec
import numpy as np

with open('pos_tweets.txt', 'r') as infile:
    pos_tweets = infile.readlines()
    
with open('neg_tweets.txt', 'r') as infile:
    neg_tweets = infile.readlines()#use 1 for positive sentiment, 0 for negative

y = np.concatenate((np.ones(len(pos_tweets)), np.zeros(len(neg_tweets))))
    
x_train, x_test, y_train, y_test = train_test_split(np.concatenate((pos_tweets, neg_tweets)), y, test_size=0.2)

def cleanText(corpus):
    corpus = [z.lower().replace('\n','').split() for z in corpus]
    return corpus

x_train = cleanText(x_train)
x_test = cleanText(x_test)

n_dim = 300

model = Word2Vec(size=n_dim, min_count=10)
model.build_vocab(x_train)#Train the model over train_reviews (this may take several minutes)
model.train(x_train,total_examples=model.corpus_count,epochs=model.epochs)

(39512, 117930)

In [2]:
#Build word vector for training set by using the average value of all word vectors in the tweet, then scale
def buildWordVector(text, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        try:
            vec += model.wv[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [3]:
from sklearn.preprocessing import scale

train_vecs = np.concatenate([buildWordVector(z, n_dim) for z in x_train])
train_vecs = scale(train_vecs)

(9603, 29755)

In [7]:
model.train(x_test,total_examples=model.corpus_count,epochs=model.epochs)
test_vecs = np.concatenate([buildWordVector(z, n_dim) for z in x_test])
test_vecs = scale(test_vecs)